In [9]:
startdate='2016-08-01'
enddate='2016-08-31'

timeinterval='({}-{})'.format(''.join(startdate.split('-')),''.join(enddate.split('-')))

In [10]:
import pandas as pd
import numpy as np
%matplotlib inline
import time
import urllib.parse
import httplib2
import json
import datetime
import os


def getQueryData(sql):
    http = httplib2.Http()
    url = 'http://ip:port/ConfComputer/sql/select.do'   
    body = {'pwd': 'eship001~', 'sql': sql}
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    response, content = http.request(url, 'POST', headers=headers, body=urllib.parse.urlencode(body))
    return content

def get(sql):
    dd=getQueryData(sql)
    df=pd.read_json(dd)
    import numpy as np
    ls=[]
    index=list(df.iloc[0,:].dropna())
    for i in range(len(df.columns)):
        if df.iat[1,i] is not None:
            ls.append(list(df.iat[1,i]))
    dt=np.array(ls).T
    ds=pd.DataFrame(index=index,data=dt).T
    return ds

data=getQueryData('show databases')
data

b'[["databaseName"],[["android"],["default"],["test"],["web_site"]]]'

In [11]:
folder='F:/sunbl_notebooks/andriodmonthreport'+timeinterval
if not os.path.exists(folder):
    os.mkdir(folder)

In [12]:
start=time.time()

In [13]:
def to_tuple_sql(app_version):
    if type(app_version) == str:
        temp=list([app_version])
        temp.append('all')
        return tuple(temp)
    if type(app_version) == list:
        return tuple(app_version)
 
def select_data(want,startdate,enddate,groupby=None,
               app_version=None,con=None,orderby=None):
    '''
    基本取数函数
    orderby='dt' or  'dt desc'
    '''
    sql='use android;'
    sql+="select {} from android_log_data".format(want)
    sql+=" where dt between '{}' and '{}'".format(startdate,enddate)
    #清洗数据
    sql+=" and app_version like '%.%.%' and length(app_version)<10"
    sql+=" and visit_path not like '-%-%'"
    #sql+=''' and visit_path != "cz-"''' #"cz-"表示为空之意
    
    if app_version is not None:
        sql+=" and app_version in {}".format(to_tuple_sql(app_version))
    
    if con is not None:
        sql+=con
        
    if groupby is not None:
        sql+=" group by {}".format(groupby)
    if orderby is not None:
        sql+=" order by {}".format(orderby)
    
    sql+=' limit 10000'
    
    start=time.time()
    df=get(sql)
    end=time.time()
    print(end-start)
    
    return df,sql

def Series_digitalize(s):
    '''
    datatype_ls=['float','int']
    '''
    ls=[]
    for j in s :
        try:
            ls.append(float(j)) 
        except:
            ls.append(j)
    Se=pd.Series(ls)
    return Se

# 指标概况

In [14]:
name='指标概况'

访问深度？这是怎么意思呢？
这个比较复杂，先放下

In [20]:
want="app_version,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits,count(launch_event),\
sum(previous_session_length) as visit_session_length"
df,sql=select_data(want,startdate,enddate,groupby='app_version',app_version=['5.6.1','5.6.0','5.5.1','5.5.0','5.4.0','5.3.0','5.2.0'])

227.5340142250061


In [ ]:
df=df.sort_values(by='app_version',ascending=False)

In [ ]:
df['人均点击']=list(Series_digitalize(df.iloc[:,2])/Series_digitalize(df.iloc[:,1]))
df['人均启动次数']=list(Series_digitalize(df.iloc[:,3])/Series_digitalize(df.iloc[:,1]))
df['人均访问时长']=list(Series_digitalize(df.iloc[:,4])/Series_digitalize(df.iloc[:,1]))

df.columns=['app_version', 'uv', 'hits', '启动次数',
       '访问时长', '人均点击', '人均启动次数', '人均访问时长']

path=folder+'/'+name+'.xlsx'
df.to_excel(path)

In [ ]:
df

# 用户规模：uv，hits

In [ ]:
name='用户规模'
name_num=0

In [ ]:
want="count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate)

In [ ]:
path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
# 月活跃，月点击
df

In [ ]:
want="dt,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate='2016-08-19',groupby='dt')

In [ ]:
path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
#日活跃，日点击
df

In [ ]:
print('日均uv：{}'.format(Series_digitalize(df.uv).mean()))
print('日均hits：{}'.format(Series_digitalize(df.hits).mean()))

In [ ]:
# 用户构成 ——此处可调整，加快速度_不能改啊
#按登录状态：电信用户免登录
#有支付
ls_data=[]

want="count(distinct tracking_id) as uv"
df,sql=select_data(want,startdate,enddate)

ls_data.append(df.iat[0,0])

want="count(distinct tracking_id) as uv"
con=" and login_status is not null"
df,sql=select_data(want,startdate,enddate,con=con)

ls_data.append(df.iat[0,0])

want="count(distinct tracking_id) as uv"
con=" and carrier_name='中国电信'"
df,sql=select_data(want,startdate,enddate,con=con)

ls_data.append(df.iat[0,0])

want="count(distinct tracking_id) as uv"
con=" and order_payment='1'"
df,sql=select_data(want,startdate,enddate,con=con)

ls_data.append(df.iat[0,0])

df=pd.DataFrame(ls_data,index=['uv','login','login_电信','有支付'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
#按使用天数状态
want="days_since_first_use,count(distinct tracking_id) as uv"

df,sql=select_data(want,startdate,enddate,groupby='days_since_first_use')

In [ ]:
df['days_since_first_use']=list(Series_digitalize(df.iloc[:,0]))

con=[]
for s in df['days_since_first_use']:
    try:
        con.append(s>=0)
    except:
        con.append(False)

dfn=df[con].sort_values(by='days_since_first_use')

In [ ]:
dfn.head(30)

In [ ]:
path=folder+'/'+name+'{}.xlsx'.format(name_num)
dfn.head(30).to_excel(path)
name_num+=1

# 版本更新用户迁移：略

# 渠道分析

In [ ]:
name='渠道分析'
name_num=0

In [ ]:
want="channels,count(distinct tracking_id) as uv,count(install_event),count(launch_event),\
sum(previous_session_length) as visit_session_length"
df,sql=select_data(want,startdate,enddate,groupby='channels')

In [ ]:
con=[s is not None for s in df.iloc[:,0]]
df=df[con]

新增用户占比：渠道份额所选时间内，所选渠道的新增用户占所有渠道新增用户总量的比例。（不含渠道None）
活跃用户占比：当前渠道活跃用户数/所有渠道活跃用户总量的比例。
新增活跃占比：当前渠道的新增用户数/当前渠道的活跃用户数的比例

In [ ]:
new_sum=Series_digitalize(df.iloc[:,2]).sum()
active_sum=Series_digitalize(df.iloc[:,1]).sum()

In [ ]:
df['新增用户占比']=list(Series_digitalize(df.iloc[:,2])/new_sum)
df['活跃用户占比']=list(Series_digitalize(df.iloc[:,1])/active_sum)
df['新增活跃占比']=list(Series_digitalize(df.iloc[:,2])/Series_digitalize(df.iloc[:,1]))
df['人均启动次数']=list(Series_digitalize(df.iloc[:,3])/Series_digitalize(df.iloc[:,1]))
df['次均使用时长']=list(Series_digitalize(df.iloc[:,4])/Series_digitalize(df.iloc[:,3]))

In [ ]:
df

In [ ]:
path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

# 留存分析:数据结果不正常，不清楚数据问题还是算法问题

app_version:5.6,5.6.1

此处计算量较大，消耗计算资源多

In [ ]:
name='留存分析'
name_num=0

0816日留存：考虑的是此日的新增用户，用的是之后的数据

In [ ]:
def newdate(dt,days):
    ndate=datetime.datetime.strptime(dt,'%Y-%m-%d')+datetime.timedelta(days)
    newdate=ndate.strftime('%Y-%m-%d')
    return newdate

In [ ]:
app_version_ls=['5.6.1','5.6.0']

want="dt,count(install_event)"
df0,sql=select_data(want,startdate,enddate,app_version=app_version_ls,groupby='dt',orderby='dt')

In [ ]:
days_ls=[0,6,13]
df_ls=[df0]
for s in days_ls:
    want="dt,count(distinct tracking_id)"
    con=" and days_since_first_use in {}".format(to_tuple_sql((str(s))))
    df,sql=select_data(want,newdate(startdate,s),newdate(enddate,s),app_version=app_version_ls,groupby='dt',con=con,orderby='dt')
    df_ls.append(df)

df_dict={'0':pd.Series(df0.iloc[:,1])}
for i,s in enumerate(days_ls):
    df_dict[str(s+1)]=pd.Series(list(df_ls[i].iloc[:,1]))

dfn=pd.DataFrame(df_dict)

dfn.index=df0['dt']



path=folder+'/'+name+'{}{}.xlsx'.format(app_version_ls,name_num)
dfn.to_excel(path)
name_num+=1

In [ ]:
dfn

####  增加留存率

# 内容分析

In [ ]:
name='内容分析'
name_num=0

In [ ]:
want="visit_path,count(distinct tracking_id) as uv,count(1) as hits"
df,sql=select_data(want,startdate,enddate,groupby='visit_path',orderby='uv desc')

df['人均点击']=list(Series_digitalize(df.iloc[:,2])/Series_digitalize(df.iloc[:,1]))

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 分版本
want="app_version,visit_path,count(distinct tracking_id) as uv,count(1) as hits"
df,sql=select_data(want,startdate,enddate,groupby='app_version,visit_path',orderby='uv desc',
                   app_version=['5.6.1','5.6.0','5.5.1','5.5.0','5.4.0','5.3.0','5.2.0'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

# 频道流量

In [ ]:
name='频道流量'
name_num=0

In [ ]:
#导航频道
want="navigation_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='navigation_click',orderby='uv desc')

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
want="app_version,navigation_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='app_version,navigation_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0','5.5.1','5.5.0'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

# 频道点击偏好分析

In [ ]:
name='频道点击偏好分析'
name_num=0

In [ ]:
#首页频道：没加‘在线时长’——不懂，没有‘按天的数据’——需要明确首页焦点图有哪些，针对取数
want="home_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='home_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])
app_version=['5.6.1','5.6.0']
path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

#首页焦点图：从上面筛选

In [ ]:
#查询办理——手机页面访客量及户均点击量
#查询办理——宽带页面访客量及户均点击量
#——没提上月数据
want="query_transact_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='query_transact_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
con=[]
for s in df.query_transact_click:
    try:
        con.append('宽带' in s)
    except:
        con.append(False)

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df[con].to_excel(path)
name_num+=1

In [ ]:
df[con]

In [ ]:
con=[]
for s in df.query_transact_click:
    try:
        con.append('手机' in s)
    except:
        con.append(False)

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df[con].to_excel(path)
name_num+=1

In [ ]:
df[con]

In [ ]:
#查询办理：主要功能点直连、非直连访问量
#直连省：安徽省','北京市','福建省','甘肃省','广东省','湖南省','江苏省','江西省','山东省','山西省','四川省','天津市','西藏省','浙江省','重庆市
#非直连省：云南省','新疆省','上海市','陕西省','青海省','宁夏省','内蒙古','辽宁省','吉林省','湖北省','黑龙江省','河南省','河北省','海南省','贵州省','广西省

zhi_ls=['安徽省','北京市','福建省','甘肃省','广东省','湖南省','江苏省','江西省','山东省','山西省','四川省','天津市','西藏省','浙江省','重庆市']
nozhi_ls=['云南省','新疆省','上海市','陕西省','青海省','宁夏省','内蒙古','辽宁省','吉林省','湖北省','黑龙江省','河南省','河北省','海南省','贵州省','广西省']

In [ ]:
#直连
want="query_transact_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
con=" and province in {}".format(to_tuple_sql(zhi_ls))
df,sql=select_data(want,startdate,enddate,groupby='query_transact_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
#非直连
want="query_transact_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
con=" and province in {}".format(to_tuple_sql(nozhi_ls))
df,sql=select_data(want,startdate,enddate,groupby='query_transact_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
#充值频道：访客量及户均点击量
want="paybtn_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='paybtn_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
#更多频道：访客量及户均点击量

want="more_click,count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits"
df,sql=select_data(want,startdate,enddate,groupby='more_click',orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])

path=folder+'/'+name+'{}.xlsx'.format(name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
def uv_hits_click(click,startdate,enddate):
    want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
    df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])
    return df,sql

In [ ]:
#我的页面频道

click='information_click'
app_version=['5.6.1','5.6.0']
df,sql=uv_hits_click(click,startdate,enddate)

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

#人均点击量，可以做个函数的

In [ ]:
# 商城频道：分类产品转化
click='product_category'
app_version=['5.6.1','5.6.0']
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 商城频道：访客量TOP20产品户均点击情况
#—— orderflow数据没加
click='product_name'
app_version=['5.6.1','5.6.0']
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'])

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 商城频道-手机类产品相关数据
click='product_name'
app_version=['5.6.1','5.6.0']
con=" and product_category='1'"
#orderflow='product_details,order_confirms,order_payment'
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
for i in [1,2,3,4,5]:
    want+=",count(case when mall_events='mall1{}' then 1 else null end) as mall1{}".format(str(i),str(i))
df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 商城频道-套餐产品相关数据 
click='product_name'
app_version=['5.6.1','5.6.0']
con=" and product_category='3'"
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
for i in [1,2,3,4,5]:
    want+=",count(case when mall_events='mall3{}' then 1 else null end) as mall3{}".format(str(i),str(i))
df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 商城频道-号码产品相关数据 
click='product_name'
app_version=['5.6.1','5.6.0']
con=" and product_category='4'"
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
for i in [1,2,3,4,5]:
    want+=",count(case when mall_events='mall4{}' then 1 else null end) as mall4{}".format(str(i),str(i))

df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
# 商城频道-套餐产品相关数据 
click='product_name'
app_version=['5.6.1','5.6.0']
con=" and product_category='5'"
want="{},count(distinct tracking_id) as uv,count(if(page_name is null,1,null)) as hits".format(click)
for i in [1,2,3,4,5]:
    want+=",count(case when mall_events='mall5{}' then 1 else null end) as mall5{}".format(str(i),str(i))

df,sql=select_data(want,startdate,enddate,groupby=click,orderby='uv desc',
                  app_version=['5.6.1','5.6.0'],con=con)

path=folder+'/'+name+'{}{}.xlsx'.format(app_version,name_num)
df.to_excel(path)
name_num+=1

In [ ]:
df

In [ ]:
end=time.time()
end-start